# **Process S1 SLC data using parallel processing**


First import all necessary libraries

In [ ]:
import ost 
import ost.helpers as h
from ost.helpers import onda, asf_wget, vector
from ost import Sentinel1_SLCBatch
import os
from os.path import join
from pathlib import Path
from pprint import pprint

Ingest shapefile data and set start and end dates

In [ ]:

# create a processing directory
project_dir = '/home/ost/Data/jwheeler/Sydney_Fires'

# apply function with buffer in meters
from ost.helpers import vector
input_shp = "/home/ost/Data/jwheeler/Shapefiles/Sydney_fires.shp"
aoi = vector.shp_to_wkt(input_shp)
#----------------------------
# Time of interest
#----------------------------
# we set only the start date to today - 30 days
start = '2019-11-30'
end = '2019-12-12'


Initiate class with above attributes

In [ ]:
# create s1Project class instance
s1_batch = Sentinel1_SLCBatch(
    project_dir=project_dir, 
    aoi=aoi, 
    start=start, 
    end=end, 
    product_type='SLC',
    ard_type='OST Plus')

Search for images on scihub and plot footprints

In [ ]:
#---------------------------------------------------
# for plotting purposes we use this iPython magic
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (19, 19)
#---------------------------------------------------

# search command
s1_batch.search()
# we plot the full Inventory on a map
s1_batch.plot_inventory(transparency=.1)

Refine image search

In [ ]:
s1_batch.refine()

Select appropriate key and plot filtered images

In [ ]:
pylab.rcParams['figure.figsize'] = (13, 13)


key = 'DESCENDING_VVVH'
s1_batch.refined_inventory_dict[key]
s1_batch.plot_inventory(s1_batch.refined_inventory_dict[key], 0.3)

Download using a selected S-1 mirror - ideally ASF (2 using request or 5 using wget) or onda (4) if accounts are set up correctly for fast, parallel downloading

In [ ]:
s1_batch.download(s1_batch.refined_inventory_dict[key],concurrent=8)

Create inventory of bursts in downloaded images, plot them and print information

In [ ]:
s1_batch.create_burst_inventory(key=key, refine=True)
pylab.rcParams['figure.figsize'] = (13, 13)
s1_batch.plot_inventory(s1_batch.burst_inventory, transparency=0.1)
print('Our burst inventory holds {} bursts to process.'.format(len(s1_batch.burst_inventory)))
print('------------------------------------------')
print(s1_batch.burst_inventory.head())

Uncomment the below command to view the current ard parameters

In [ ]:
#pprint(s1_batch.ard_parameters)

Run the s1SLCbatch class function bursts to ard to generate parameter text files for each step from burst to ard, ard to timeseries, timeseries to timescan and mosaic.

**NB Use a base name for the exec file without a extension AND make sure to choose the number of cores that each process will use for parallel processing. ncores in this function x multiproc in the multiprocess function should not exceed the number of cores on your machine**

In [ ]:
s1_batch.bursts_to_ard(timeseries=True, timescan=True, mosaic=True, overwrite=False, exec_file='/home/ost/Data/jwheeler/Sydney_Fires/test', ncores=2)
#print(s1_batch.temp_dir)

Run the s1SLCbatch class function multiprocessing to run, sequentially, the parameters in the previously generated text files for each step from burst to ard, ard to timeseries, timeseries to timescan and mosaic.

**NB Use the same base name for the exec file without a extension as before AND make sure to choose the number of cores that each process will use for parallel processing as well as the number of concurrent processes. ncores in the previous function x multiproc in this function should not exceed the number of cores on your machine. You should also include the ncores again as the text file generation is reiterated during this process**

In [ ]:
s1_batch.multiprocess(timeseries=True, timescan=True, mosaic=True, overwrite=False, exec_file='/home/ost/Data/jwheeler/Sydney_Fires/test', ncores=2,multiproc=4)
#burst_to_ard_batch(s1_batch.burst_inventory,s1_batch.download_dir,s1_batch.processing_dir,s1_batch.temp_dir,s1_batch.proc_file,exec_file='/home/ost/Data/jwheeler/Sydney_Fires/test.txt')